<a href="https://colab.research.google.com/github/R3gm/SD_diffusers_interactive/blob/main/Stable_diffusion_interactive_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Interactive Notebook 📓 🤖

| Description | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/github/stars/R3gm/SD_diffusers_interactive?style=social)](https://github.com/R3gm/SD_diffusers_interactive) |


- Compel added for SD 1.5 (prompt weights)
- Controlnet 1.1 for SD
- SDXL models only support txt2img
- Lora usually doesn't work correctly alongside Controlnet
- More functions, more bugs; less than 10 words, more laughs




A widgets-based interactive notebook for Google Colab that lets users generate AI images from prompts (Text2Image) using [Stable Diffusion (by Stability AI, Runway & CompVis)](https://en.wikipedia.org/wiki/Stable_Diffusion).

This notebook aims to be an alternative to WebUIs while offering a simple and lightweight GUI for `anyone to get started with Stable Diffusion`.

Uses Stable Diffusion, [HuggingFace](https://huggingface.co/) Diffusers and [Jupyter widgets](https://github.com/jupyter-widgets/ipywidgets).

<br/>

Based on redromnon's repository

[Original GitHub](https://github.com/redromnon/stable-diffusion-interactive-notebook)

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown
#@markdown ---

!pip install -q omegaconf==2.3.0 torch git+https://github.com/huggingface/diffusers.git git+https://github.com/damian0815/compel.git invisible_watermark  transformers accelerate scipy safetensors==0.3.3 xformers safetensors mediapy ipywidgets==7.7.1 controlnet_aux==0.0.6 mediapipe==0.10.1 pytorch-lightning asdff

!apt install git-lfs
!git lfs install
!apt -y install -qq aria2

`RESTART THE RUNTIME` before executing the next cell.

In [ ]:
#@title 👇 Download Model: Please provide a link for the Civitai API, Google Drive, or Hugging Face. { form-width: "20%", display-mode: "form" }
import os
%cd /content

def download_things(directory, url, hf_token=""):
    url = url.strip()

    if "drive.google.com" in url:
        original_dir = os.getcwd()
        os.chdir(directory)
        !gdown --fuzzy {url}
        os.chdir(original_dir)
    elif "huggingface.co" in url:
        if "/blob/" in url:
            url = url.replace("/blob/", "/resolve/")
        user_header = f'"Authorization: Bearer {hf_token}"'
        !aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 {url} -d {directory}  -o {url.split('/')[-1]}
    else:
        !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {directory} {url}

def get_model_list(directory_path):
    model_list = []
    valid_extensions = {'.ckpt' , '.pt', '.pth', '.safetensors', '.bin'}

    for filename in os.listdir(directory_path):
        if os.path.splitext(filename)[1] in valid_extensions:
            name_without_extension = os.path.splitext(filename)[0]
            file_path = os.path.join(directory_path, filename)
            model_list.append((name_without_extension, file_path))
            print('\033[34mFILE: ' + name_without_extension + file_path + '\033[0m')
    return model_list

def process_string(input_string):
    parts = input_string.split('/')

    if len(parts) == 2:
        first_element = parts[1]
        complete_string = input_string
        result = (first_element, complete_string)
        return result
    else:
        return None

directory_models = 'models'
os.makedirs(directory_models, exist_ok=True)
directory_loras = 'loras'
os.makedirs(directory_loras, exist_ok=True)
directory_vaes = 'vaes'
os.makedirs(directory_vaes, exist_ok=True)

#@markdown ---
#@markdown - **Download a Model**
download_model = "https://civitai.com/api/download/models/125771" # @param {type:"string"}
#@markdown - For SDXL models, only diffuser format models are supported, and you only need the repository name.
load_diffusers_format_model = 'Linaqruf/animagine-xl' # @param {type:"string"}
#@markdown - **Download a VAE**
download_vae = "https://huggingface.co/WarriorMama777/OrangeMixs/blob/main/VAEs/orangemix.vae.pt" # @param {type:"string"}
#@markdown - **Download a LoRA**
download_lora = "https://civitai.com/api/download/models/97655" # @param {type:"string"}
#@markdown ---
#@markdown **HF TOKEN** - If you need to download your private model from Hugging Face, input your token here.
hf_token = ""  # @param {type:"string"}
#@markdown
#@markdown ---

download_things(directory_models, download_model, hf_token)
download_things(directory_vaes, download_vae, hf_token)
download_things(directory_loras, download_lora, hf_token)


# TI more combatible in safetensor format; maybe convert to safetensor can help
directory_embeds = 'embedings'
os.makedirs(directory_embeds, exist_ok=True)
download_embeds = [
    'https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt.pt',
    'https://huggingface.co/datasets/Nerfgun3/bad_prompt/blob/main/bad_prompt_version2.pt',
    'https://huggingface.co/sayakpaul/EasyNegative-test/blob/main/EasyNegative.safetensors',
    ]
for url_embed in download_embeds:
    download_things(directory_embeds, url_embed, hf_token)
embed_list = get_model_list(directory_embeds)

model_list = get_model_list(directory_models)
if load_diffusers_format_model.strip() != "" and load_diffusers_format_model.count('/') == 1:
    model_list.append(process_string(load_diffusers_format_model))
lora_model_list = get_model_list(directory_loras)
lora_model_list.insert(0, ("None",None))
vae_model_list = get_model_list(directory_vaes)
vae_model_list.insert(0, ("None","None"))



print('\033[33m🏁 Download finished.\033[0m')

### SECOND PART ###
import gc
import time
import numpy as np
import PIL.Image
from diffusers import (
    ControlNetModel,
    DiffusionPipeline,
    StableDiffusionControlNetPipeline,
    StableDiffusionControlNetInpaintPipeline,
    StableDiffusionPipeline,
    AutoencoderKL
)
import torch
import random
from controlnet_aux import (CannyDetector, ContentShuffleDetector, HEDdetector,
                            LineartAnimeDetector, LineartDetector,
                            MidasDetector, MLSDdetector, NormalBaeDetector,
                            OpenposeDetector, PidiNetDetector)
from transformers import pipeline
from controlnet_aux.util import HWC3, ade_palette
from transformers import AutoImageProcessor, UperNetForSemanticSegmentation
import cv2
from diffusers import (
    DPMSolverMultistepScheduler,
    DPMSolverSinglestepScheduler,
    KDPM2DiscreteScheduler,
    KDPM2AncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    HeunDiscreteScheduler,
    LMSDiscreteScheduler,
    DDIMScheduler,
    DEISMultistepScheduler,
    UniPCMultistepScheduler,
)

# =====================================
# Utils preprocessor
# =====================================
def resize_image(input_image, resolution, interpolation=None):
    H, W, C = input_image.shape
    H = float(H)
    W = float(W)
    k = float(resolution) / max(H, W)
    H *= k
    W *= k
    H = int(np.round(H / 64.0)) * 64
    W = int(np.round(W / 64.0)) * 64
    if interpolation is None:
        interpolation = cv2.INTER_LANCZOS4 if k > 1 else cv2.INTER_AREA
    img = cv2.resize(input_image, (W, H), interpolation=interpolation)
    return img

class DepthEstimator:
    def __init__(self):
        self.model = pipeline('depth-estimation')

    def __call__(self, image: np.ndarray, **kwargs) -> PIL.Image.Image:
        detect_resolution = kwargs.pop('detect_resolution', 512)
        image_resolution = kwargs.pop('image_resolution', 512)
        image = np.array(image)
        image = HWC3(image)
        image = resize_image(image, resolution=detect_resolution)
        image = PIL.Image.fromarray(image)
        image = self.model(image)
        image = image['depth']
        image = np.array(image)
        image = HWC3(image)
        image = resize_image(image, resolution=image_resolution)
        return PIL.Image.fromarray(image)

class ImageSegmentor:
    def __init__(self):
        self.image_processor = AutoImageProcessor.from_pretrained(
            'openmmlab/upernet-convnext-small')
        self.image_segmentor = UperNetForSemanticSegmentation.from_pretrained(
            'openmmlab/upernet-convnext-small')

    @torch.inference_mode()
    def __call__(self, image: np.ndarray, **kwargs) -> PIL.Image.Image:
        detect_resolution = kwargs.pop('detect_resolution', 512)
        image_resolution = kwargs.pop('image_resolution', 512)
        image = HWC3(image)
        image = resize_image(image, resolution=detect_resolution)
        image = PIL.Image.fromarray(image)

        pixel_values = self.image_processor(image,
                                            return_tensors='pt').pixel_values
        outputs = self.image_segmentor(pixel_values)
        seg = self.image_processor.post_process_semantic_segmentation(
            outputs, target_sizes=[image.size[::-1]])[0]
        color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
        for label, color in enumerate(ade_palette()):
            color_seg[seg == label, :] = color
        color_seg = color_seg.astype(np.uint8)

        color_seg = resize_image(color_seg,
                                 resolution=image_resolution,
                                 interpolation=cv2.INTER_NEAREST)
        return PIL.Image.fromarray(color_seg)

class Preprocessor:
    MODEL_ID = 'lllyasviel/Annotators'

    def __init__(self):
        self.model = None
        self.name = ''

    def load(self, name: str) -> None:
        if name == self.name:
            return
        if name == 'HED':
            self.model = HEDdetector.from_pretrained(self.MODEL_ID)
        elif name == 'Midas':
            self.model = MidasDetector.from_pretrained(self.MODEL_ID)
        elif name == 'MLSD':
            self.model = MLSDdetector.from_pretrained(self.MODEL_ID)
        elif name == 'Openpose':
            self.model = OpenposeDetector.from_pretrained(self.MODEL_ID)
        elif name == 'PidiNet':
            self.model = PidiNetDetector.from_pretrained(self.MODEL_ID)
        elif name == 'NormalBae':
            self.model = NormalBaeDetector.from_pretrained(self.MODEL_ID)
        elif name == 'Lineart':
            self.model = LineartDetector.from_pretrained(self.MODEL_ID)
        elif name == 'LineartAnime':
            self.model = LineartAnimeDetector.from_pretrained(self.MODEL_ID)
        elif name == 'Canny':
            self.model = CannyDetector()
        elif name == 'ContentShuffle':
            self.model = ContentShuffleDetector()
        elif name == 'DPT':
            self.model = DepthEstimator()
        elif name == 'UPerNet':
            self.model = ImageSegmentor()
        else:
            raise ValueError
        torch.cuda.empty_cache()
        gc.collect()
        self.name = name

    def __call__(self, image: PIL.Image.Image, **kwargs) -> PIL.Image.Image:
        if self.name == 'Canny':
            if 'detect_resolution' in kwargs:
                detect_resolution = kwargs.pop('detect_resolution')
                image = np.array(image)
                image = HWC3(image)
                image = resize_image(image, resolution=detect_resolution)
            image = self.model(image, **kwargs)
            return PIL.Image.fromarray(image)
        elif self.name == 'Midas':
            detect_resolution = kwargs.pop('detect_resolution', 512)
            image_resolution = kwargs.pop('image_resolution', 512)
            image = np.array(image)
            image = HWC3(image)
            image = resize_image(image, resolution=detect_resolution)
            image = self.model(image, **kwargs)
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            return PIL.Image.fromarray(image)
        else:
            return self.model(image, **kwargs)


# =====================================
# Base Model
# =====================================

CONTROLNET_MODEL_IDS = {
    'Openpose': 'lllyasviel/control_v11p_sd15_openpose',
    'Canny': 'lllyasviel/control_v11p_sd15_canny',
    'MLSD': 'lllyasviel/control_v11p_sd15_mlsd',
    'scribble': 'lllyasviel/control_v11p_sd15_scribble',
    'softedge': 'lllyasviel/control_v11p_sd15_softedge',
    'segmentation': 'lllyasviel/control_v11p_sd15_seg',
    'depth': 'lllyasviel/control_v11f1p_sd15_depth',
    'NormalBae': 'lllyasviel/control_v11p_sd15_normalbae',
    'lineart': 'lllyasviel/control_v11p_sd15_lineart',
    'lineart_anime': 'lllyasviel/control_v11p_sd15s2_lineart_anime',
    'shuffle': 'lllyasviel/control_v11e_sd15_shuffle',
    'ip2p': 'lllyasviel/control_v11e_sd15_ip2p',
    'Inpaint': 'lllyasviel/control_v11p_sd15_inpaint',
    'txt2img': 'NotControlnet',
}

def download_all_controlnet_weights() -> None:
    for model_id in CONTROLNET_MODEL_IDS.values():
        ControlNetModel.from_pretrained(model_id)

class Model:
    def __init__(self,
                 base_model_id: str = 'runwayml/stable-diffusion-v1-5',
                 task_name: str = 'Canny', vae_model=None, type_model_precision = torch.float16):
        self.device = torch.device(
            'cuda:0' if torch.cuda.is_available() else 'cpu')
        self.base_model_id = ''
        self.task_name = ''
        self.vae_model = None
        self.type_model_precision = type_model_precision if torch.cuda.is_available() else torch.float32 # For SD 1.5

        self.pipe = self.load_pipe(base_model_id, task_name, vae_model, type_model_precision)
        self.preprocessor = Preprocessor()

    def load_pipe(self, base_model_id: str, task_name, vae_model=None, type_model_precision = torch.float16, reload=False) -> DiffusionPipeline:
        if base_model_id == self.base_model_id and task_name == self.task_name and hasattr(
                self, 'pipe') and self.vae_model==vae_model and self.pipe is not None and reload==False and type_model_precision == self.type_model_precision:
            print('Previous loaded')
            return self.pipe
        if base_model_id == self.base_model_id and task_name == self.task_name and hasattr(
                self, 'pipe') and self.vae_model==vae_model and self.pipe is not None and reload==False and self.device == "cpu":
            print('Pipe in CPU')
            return self.pipe


        self.type_model_precision = type_model_precision if torch.cuda.is_available() else torch.float32
        self.pipe = None
        torch.cuda.empty_cache()
        gc.collect()

        model_id = CONTROLNET_MODEL_IDS[task_name]

        if task_name == 'txt2img':
            if os.path.exists(base_model_id):
                if self.type_model_precision == torch.float32:
                    print("Working with full precision")
                pipe = StableDiffusionPipeline.from_single_file(
                  base_model_id,
                  vae = None if vae_model == 'None' else AutoencoderKL.from_single_file(vae_model), # , torch_dtype=self.type_model_precision
                  torch_dtype=self.type_model_precision,
                )
                pipe.safety_checker = None
            else:
                print("Default VAE: madebyollin/sdxl-vae-fp16-fix")
                pipe = DiffusionPipeline.from_pretrained(
                    base_model_id,
                    vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16),
                    torch_dtype=torch.float16,
                    use_safetensors=True,
                    variant="fp16",
                    )
                pipe.safety_checker = None
            print('Loaded txt2img pipeline')
        elif task_name == 'Inpaint':
            if self.type_model_precision == torch.float32:
                print("Working with full precision")
            controlnet = ControlNetModel.from_pretrained(model_id,
                                                        torch_dtype=self.type_model_precision)
            if os.path.exists(base_model_id):
                pipe = StableDiffusionControlNetInpaintPipeline.from_single_file(
                    base_model_id,
                    vae = None if vae_model == 'None' else AutoencoderKL.from_single_file(vae_model),
                    safety_checker=None,
                    controlnet=controlnet,
                    torch_dtype=self.type_model_precision)
            print('Loaded ControlNet Inpaint pipeline')
        else:
            if self.type_model_precision == torch.float32:
                print("Working with full precision")
            controlnet = ControlNetModel.from_pretrained(model_id,
                                                        torch_dtype=self.type_model_precision) # for all
            if os.path.exists(base_model_id):
                pipe = StableDiffusionControlNetPipeline.from_single_file(
                    base_model_id,
                    vae = None if vae_model == 'None' else AutoencoderKL.from_single_file(vae_model),
                    safety_checker=None,
                    controlnet=controlnet,
                    torch_dtype=self.type_model_precision)
            else:
                raise ZeroDivisionError("Not implemented")
            #     pipe = StableDiffusionControlNetPipeline.from_pretrained(
            #         base_model_id,
            #         vae = AutoencoderKL.from_pretrained(base_model_id, subfolder='vae') if vae_model == 'None' else AutoencoderKL.from_single_file(vae_model),
            #         safety_checker=None,
            #         controlnet=controlnet,
            #         torch_dtype=torch.float16)
            # print('Loaded ControlNet pipeline')

            pipe.scheduler = UniPCMultistepScheduler.from_config(
                pipe.scheduler.config)

        if self.device.type == 'cuda':
            pipe.enable_xformers_memory_efficient_attention()

        pipe.to(self.device)
        torch.cuda.empty_cache()
        gc.collect()
        self.pipe = pipe
        self.base_model_id = base_model_id
        self.task_name = task_name
        self.vae_model = vae_model
        return pipe

    def set_base_model(self, base_model_id: str) -> str:
        if not base_model_id or base_model_id == self.base_model_id:
            return self.base_model_id
        del self.pipe
        torch.cuda.empty_cache()
        gc.collect()
        try:
            self.pipe = self.load_pipe(base_model_id, self.task_name, self.vae_model)
        except Exception:
            self.pipe = self.load_pipe(self.base_model_id, self.task_name, self.vae_model)
        return self.base_model_id

    def load_controlnet_weight(self, task_name: str) -> None:
        if task_name == self.task_name:
            return
        if self.pipe is not None and hasattr(self.pipe, 'controlnet'):
            del self.pipe.controlnet
        torch.cuda.empty_cache()
        gc.collect()
        model_id = CONTROLNET_MODEL_IDS[task_name]
        controlnet = ControlNetModel.from_pretrained(model_id,
                                                     torch_dtype=self.type_model_precision)
        controlnet.to(self.device)
        torch.cuda.empty_cache()
        gc.collect()
        self.pipe.controlnet = controlnet
        self.task_name = task_name

    def get_prompt(self, prompt: str, additional_prompt: str) -> str:
        if not prompt:
            prompt = additional_prompt
        else:
            prompt = f'{prompt}, {additional_prompt}'
        return prompt

    @torch.autocast('cuda')
    def run_pipe(
        self,
        prompt: str,
        negative_prompt: str,
        prompt_embeds,
        negative_prompt_embeds,
        control_image: PIL.Image.Image,
        num_images: int,
        num_steps: int,
        guidance_scale: float,
        clip_skip: int,
        generator,
        controlnet_conditioning_scale,
        control_guidance_start,
        control_guidance_end,
    ) -> list[PIL.Image.Image]:
        # Return PIL images
        #generator = torch.Generator().manual_seed(seed)
        return self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
            guidance_scale=guidance_scale,
            clip_skip = clip_skip,
            num_images_per_prompt=num_images,
            num_inference_steps=num_steps,
            generator=generator,
            controlnet_conditioning_scale = controlnet_conditioning_scale,
            control_guidance_start = control_guidance_start,
            control_guidance_end = control_guidance_end,
            image=control_image
            ).images

    @torch.autocast('cuda')
    def run_pipe_SD(
        self,
        prompt: str,
        negative_prompt: str,
        prompt_embeds,
        negative_prompt_embeds,
        num_images: int,
        num_steps: int,
        guidance_scale: float,
        clip_skip: int,
        height : int,
        width : int,
        generator,
    ) -> list[PIL.Image.Image]:
        # Return PIL images
        #generator = torch.Generator().manual_seed(seed)
        return self.pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
            guidance_scale=guidance_scale,
            clip_skip = clip_skip,
            num_images_per_prompt=num_images,
            num_inference_steps=num_steps,
            generator=generator,
            height = height,
            width = width,
            ).images


    # @torch.autocast('cuda')
    # def run_pipe_SDXL(
    #     self,
    #     prompt: str,
    #     negative_prompt: str,
    #     prompt_embeds,
    #     negative_prompt_embeds,
    #     num_images: int,
    #     num_steps: int,
    #     guidance_scale: float,
    #     clip_skip: int,
    #     height : int,
    #     width : int,
    #     generator,
    #     seddd,
    #     conditioning,
    #     pooled,
    # ) -> list[PIL.Image.Image]:
    #     # Return PIL images
    #     #generator = torch.Generator("cuda").manual_seed(seddd) # generator = torch.Generator("cuda").manual_seed(seed),
    #     return self.pipe(
    #         prompt = None,
    #         negative_prompt = None,
    #         prompt_embeds=conditioning[0:1],
    #         pooled_prompt_embeds=pooled[0:1],
    #         negative_prompt_embeds=conditioning[1:2],
    #         negative_pooled_prompt_embeds=pooled[1:2],
    #         height = height,
    #         width = width,
    #         num_inference_steps = num_steps,
    #         guidance_scale = guidance_scale,
    #         clip_skip = clip_skip,
    #         num_images_per_prompt = num_images,
    #         generator = generator,
    #         ).images

    @torch.autocast('cuda')
    def run_pipe_inpaint(
        self,
        prompt: str,
        negative_prompt: str,
        prompt_embeds,
        negative_prompt_embeds,
        control_image: PIL.Image.Image,
        num_images: int,
        num_steps: int,
        guidance_scale: float,
        clip_skip: int,
        strength: float,
        init_image,
        control_mask,
        controlnet_conditioning_scale,
        control_guidance_start,
        control_guidance_end,
        generator,
    ) -> list[PIL.Image.Image]:
        # Return PIL images
        #generator = torch.Generator().manual_seed(seed)
        return self.pipe(
            prompt= None,
            negative_prompt= None,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
            eta=1.0,
            strength = strength,
            image = init_image, # original image
            mask_image = control_mask, # mask, values of 0 to 255
            control_image = control_image, # tensor control image
            num_images_per_prompt  = num_images,
            num_inference_steps = num_steps,
            guidance_scale = guidance_scale,
            clip_skip = clip_skip,
            generator = generator,
            controlnet_conditioning_scale = controlnet_conditioning_scale,
            control_guidance_start = control_guidance_start,
            control_guidance_end = control_guidance_end,
            ).images

    ### self.x_process return image_preprocessor###
    @torch.inference_mode()
    def process_canny(
        self,
        image: np.ndarray,
        image_resolution: int = 512,
        low_threshold: int = 100,
        high_threshold: int = 200,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        self.preprocessor.load('Canny')
        control_image = self.preprocessor(image=image,
                                          low_threshold=low_threshold,
                                          high_threshold=high_threshold,
                                          detect_resolution=image_resolution)

        self.load_controlnet_weight('Canny')

        return control_image

    @torch.inference_mode()
    def process_mlsd(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        value_threshold: float,
        distance_threshold: float,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        self.preprocessor.load('MLSD')
        control_image = self.preprocessor(
            image=image,
            image_resolution=image_resolution,
            detect_resolution=preprocess_resolution,
            thr_v=value_threshold,
            thr_d=distance_threshold,
        )
        self.load_controlnet_weight('MLSD')

        return control_image

    @torch.inference_mode()
    def process_scribble(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        elif preprocessor_name == 'HED':
            self.preprocessor.load(preprocessor_name)
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                scribble=False,
            )
        elif preprocessor_name == 'PidiNet':
            self.preprocessor.load(preprocessor_name)
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                safe=False,
            )
        self.load_controlnet_weight('scribble')

        return control_image

    @torch.inference_mode()
    def process_scribble_interactive(
        self,
        image_and_mask: dict[str, np.ndarray],
        image_resolution: int,
    ) -> list[PIL.Image.Image]:
        if image_and_mask is None:
            raise ValueError

        image = image_and_mask['mask']
        image = HWC3(image)
        image = resize_image(image, resolution=image_resolution)
        control_image = PIL.Image.fromarray(image)

        self.load_controlnet_weight('scribble')

        return control_image

    @torch.inference_mode()
    def process_softedge(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        elif preprocessor_name in ['HED', 'HED safe']:
            safe = 'safe' in preprocessor_name
            self.preprocessor.load('HED')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                scribble=safe,
            )
        elif preprocessor_name in ['PidiNet', 'PidiNet safe']:
            safe = 'safe' in preprocessor_name
            self.preprocessor.load('PidiNet')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                safe=safe,
            )
        else:
            raise ValueError
        self.load_controlnet_weight('softedge')

        return control_image

    @torch.inference_mode()
    def process_openpose(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        else:
            self.preprocessor.load('Openpose')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                hand_and_face=True,
            )
        self.load_controlnet_weight('Openpose')

        return control_image

    @torch.inference_mode()
    def process_segmentation(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        else:
            self.preprocessor.load(preprocessor_name)
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
            )
        self.load_controlnet_weight('segmentation')

        return control_image

    @torch.inference_mode()
    def process_depth(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        else:
            self.preprocessor.load(preprocessor_name)
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
            )
        self.load_controlnet_weight('depth')

        return control_image

    @torch.inference_mode()
    def process_normal(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        else:
            self.preprocessor.load('NormalBae')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
            )
        self.load_controlnet_weight('NormalBae')

        return control_image

    @torch.inference_mode()
    def process_lineart(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name in ['None', 'None (anime)']:
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        elif preprocessor_name in ['Lineart', 'Lineart coarse']:
            coarse = 'coarse' in preprocessor_name
            self.preprocessor.load('Lineart')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
                coarse=coarse,
            )
        elif preprocessor_name == 'Lineart (anime)':
            self.preprocessor.load('LineartAnime')
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
                detect_resolution=preprocess_resolution,
            )
        if 'anime' in preprocessor_name:
            self.load_controlnet_weight('lineart_anime')
        else:
            self.load_controlnet_weight('lineart')

        return control_image

    @torch.inference_mode()
    def process_shuffle(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocessor_name: str,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        if preprocessor_name == 'None':
            image = HWC3(image)
            image = resize_image(image, resolution=image_resolution)
            control_image = PIL.Image.fromarray(image)
        else:
            self.preprocessor.load(preprocessor_name)
            control_image = self.preprocessor(
                image=image,
                image_resolution=image_resolution,
            )
        self.load_controlnet_weight('shuffle')

        return control_image

    @torch.inference_mode()
    def process_ip2p(
        self,
        image: np.ndarray,
        image_resolution: int,
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        image = HWC3(image)
        image = resize_image(image, resolution=image_resolution)
        control_image = PIL.Image.fromarray(image)
        self.load_controlnet_weight('ip2p')

        return control_image

    @torch.inference_mode()
    def process_inpaint(
        self,
        image: np.ndarray,
        image_resolution: int,
        preprocess_resolution: int,
        image_mask: str,###
    ) -> list[PIL.Image.Image]:
        if image is None:
            raise ValueError

        image = HWC3(image)
        image = resize_image(image, resolution=preprocess_resolution)
        init_image = PIL.Image.fromarray(image)

        image_mask = HWC3(image_mask)
        image_mask = resize_image(image_mask, resolution=preprocess_resolution)
        control_mask = PIL.Image.fromarray(image_mask)

        control_image = make_inpaint_condition(init_image, control_mask)

        self.load_controlnet_weight('Inpaint')

        return init_image, control_mask, control_image

    def get_scheduler(self, name):
      #Get scheduler
      match name:

        case "DPM++ 2M":
          return DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config)

        case "DPM++ 2M Karras":
          return DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config, use_karras_sigmas=True)

        case "DPM++ 2M SDE":
          return DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config, algorithm_type="sde-dpmsolver++")

        case "DPM++ 2M SDE Karras":
          return DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config, use_karras_sigmas=True, algorithm_type="sde-dpmsolver++")

        case "DPM++ SDE":
          return DPMSolverSinglestepScheduler.from_config(self.pipe.scheduler.config, )

        case "DPM++ SDE Karras":
          return DPMSolverSinglestepScheduler.from_config(self.pipe.scheduler.config, use_karras_sigmas=True)

        case "DPM2":
          return KDPM2DiscreteScheduler.from_config(self.pipe.scheduler.config, )

        case "DPM2 Karras":
          return KDPM2DiscreteScheduler.from_config(self.pipe.scheduler.config, use_karras_sigmas=True)

        case "Euler":
          return EulerDiscreteScheduler.from_config(self.pipe.scheduler.config, )

        case "Euler a":
          return EulerAncestralDiscreteScheduler.from_config(self.pipe.scheduler.config, )

        case "Heun":
          return HeunDiscreteScheduler.from_config(self.pipe.scheduler.config, )

        case "LMS":
          return LMSDiscreteScheduler.from_config(self.pipe.scheduler.config, )

        case "LMS Karras":
          return LMSDiscreteScheduler.from_config(self.pipe.scheduler.config, use_karras_sigmas=True)

        case "DDIMScheduler":
          return DDIMScheduler.from_config(self.pipe.scheduler.config)

        case "DEISMultistepScheduler":
          return DEISMultistepScheduler.from_config(self.pipe.scheduler.config)

        case "UniPCMultistepScheduler":
          return UniPCMultistepScheduler.from_config(self.pipe.scheduler.config)

    def process_lora(self, select_lora, lora_weights_scale, unload=False):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if not unload:
            if select_lora != None:
                  try:
                      self.pipe = lora_mix_load(self.pipe, select_lora, lora_weights_scale, device=device, dtype=self.type_model_precision)
                      print(select_lora)
                  except:
                      print(f"ERROR: LoRA not compatible: {select_lora}")
            return self.pipe
        else:
            # Unload numerically unstable but fast
            if select_lora != None:
                try:
                    self.pipe = lora_mix_load(self.pipe, select_lora, -lora_weights_scale, device=device, dtype=self.type_model_precision)
                    #print(select_lora, 'unload')
                except:
                    pass
            return self.pipe
    def __call__(
        self,
        prompt = None,
        negative_prompt = None,
        # prompt_embeds = None,
        # negative_prompt_embeds = None,
        img_height = None,
        img_width = None,
        num_images = None,
        num_steps = None,
        guidance_scale = None,
        clip_skip = True,
        seed = None,
        image = None, # path, np.array, or PIL image
        preprocessor_name = None,
        preprocess_resolution = None,
        image_resolution = None,
        additional_prompt = "",
        image_mask = None,
        strength = None,
        low_threshold=None, # Canny
        high_threshold=None, # Canny
        value_threshold=None, # MLSD
        distance_threshold=None, # MLSD
        lora_A = None,
        lora_scale_A = 1.0,
        lora_B = None,
        lora_scale_B = 1.0,
        lora_C = None,
        lora_scale_C = 1.0,
        lora_D = None,
        lora_scale_D = 1.0,
        lora_E = None,
        lora_scale_E = 1.0,
        active_textual_inversion = False,
        textual_inversion = [], # List of tuples [(activation_token, path_embedding),...]
        convert_weights_prompt = False,
        sampler = None,
        xformers_memory_efficient_attention = True,
        gui_active = False,
        loop_generation = 1,
        controlnet_conditioning_scale = 1.0,
        control_guidance_start = 0.0,
        control_guidance_end  = 0.0,
        generator_in_cpu = False, # Initial noise not in CPU
    ):
        if self.task_name != "txt2img" and image == None:
            raise ValueError

        if self.pipe == None:
            self.load_pipe(self.base_model_id, task_name = self.task_name, vae_model = self.vae_model)

        self.pipe.to(self.device)
        # self.pipe.unfuse_lora()
        # self.pipe.unload_lora_weights()
        self.pipe = self.process_lora(lora_A, lora_scale_A)
        self.pipe = self.process_lora(lora_B, lora_scale_B)
        self.pipe = self.process_lora(lora_C, lora_scale_C)
        self.pipe = self.process_lora(lora_D, lora_scale_D)
        self.pipe = self.process_lora(lora_E, lora_scale_E)
        self.pipe.to(self.device)

        # Prompt Optimizations for 1.5
        if os.path.exists(self.base_model_id):
            if  active_textual_inversion:
              # Textual Inversion
              for name, directory_name in textual_inversion:
                  try:
                          #self.pipe.text_encoder.resize_token_embeddings(len(self.pipe.tokenizer),pad_to_multiple_of=128)
                          #self.pipe.load_textual_inversion("./bad_prompt.pt", token="baddd")
                          self.pipe.load_textual_inversion(directory_name, token=name)
                  except ValueError:
                      # previous loaded ti
                      pass
                  except:
                      print(f"Can't apply {name}")

            #Clip skip
            if clip_skip:
                #clip_skip_diffusers = None #clip_skip - 1 # future update
                compel = Compel(
                    tokenizer=self.pipe.tokenizer,
                    text_encoder=self.pipe.text_encoder,
                    truncate_long_prompts=False,
                    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NORMALIZED
                )
            else:
                #clip_skip_diffusers = None # clip_skip = None # future update
                compel = Compel(
                    tokenizer=self.pipe.tokenizer,
                    text_encoder=self.pipe.text_encoder,
                    truncate_long_prompts=False
                )

            # Prompt weights for textual inversion
            prompt_ti = self.pipe.maybe_convert_prompt(prompt, self.pipe.tokenizer)
            negative_prompt_ti = self.pipe.maybe_convert_prompt(negative_prompt, self.pipe.tokenizer)

            # prompt syntax style a1...
            if convert_weights_prompt:
                prompt_ti = prompt_weight_conversor(prompt_ti)
                negative_prompt_ti = prompt_weight_conversor(negative_prompt_ti)

            # prompt embed chunks style a1...
            prompt_emb = merge_embeds(tokenize_line(prompt_ti, self.pipe.tokenizer), compel)
            negative_prompt_emb = merge_embeds(tokenize_line(negative_prompt_ti, self.pipe.tokenizer), compel)

            # fix error shape
            if prompt_emb.shape != negative_prompt_emb.shape:
                prompt_emb, negative_prompt_emb = compel.pad_conditioning_tensors_to_same_length([prompt_emb, negative_prompt_emb])

            compel = None
            del compel

        # Prompt Optimizations for SDXL
        else:
            if  active_textual_inversion:
              # Textual Inversion
              # for name, directory_name in textual_inversion:
              #     try:
              #             #self.pipe.text_encoder.resize_token_embeddings(len(self.pipe.tokenizer),pad_to_multiple_of=128)
              #             #self.pipe.load_textual_inversion("./bad_prompt.pt", token="baddd")
              #             self.pipe.load_textual_inversion(directory_name, token=name)
              #     except ValueError:
              #         # previous loaded ti
              #         pass
              #     except:
              #         print(f"Can't apply {name}")
              print("SDXL textual inversion not available")

            #Clip skip
            if clip_skip:
                #clip_skip_diffusers = None #clip_skip - 1 # future update
                compel = Compel(
                    tokenizer=[self.pipe.tokenizer, self.pipe.tokenizer_2] ,
                    text_encoder=[self.pipe.text_encoder, self.pipe.text_encoder_2],
                    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
                    requires_pooled=[False, True],
                    truncate_long_prompts=False)
            else:
                #clip_skip_diffusers = None # clip_skip = None # future update
                compel = Compel(
                    tokenizer=[self.pipe.tokenizer, self.pipe.tokenizer_2] ,
                    text_encoder=[self.pipe.text_encoder, self.pipe.text_encoder_2],
                    requires_pooled=[False, True],
                    truncate_long_prompts=False)

            # Prompt weights for textual inversion
            # prompt_ti = self.pipe.maybe_convert_prompt(prompt, self.pipe.tokenizer)
            # negative_prompt_ti = self.pipe.maybe_convert_prompt(negative_prompt, self.pipe.tokenizer)

            # prompt syntax style a1...
            if convert_weights_prompt:
                prompt_ti = prompt_weight_conversor(prompt)
                negative_prompt_ti = prompt_weight_conversor(negative_prompt)
            else:
                prompt_ti = prompt
                negative_prompt_ti = negative_prompt

            # prompt embed chunks style a1...
            # prompt_emb = merge_embeds(tokenize_line(prompt_ti, self.pipe.tokenizer), compel)
            # negative_prompt_emb = merge_embeds(tokenize_line(negative_prompt_ti, self.pipe.tokenizer), compel)

            # fix error shape
            # if prompt_emb.shape != negative_prompt_emb.shape:
            #     prompt_emb, negative_prompt_emb = compel.pad_conditioning_tensors_to_same_length([prompt_emb, negative_prompt_emb])

            conditioning, pooled = compel([prompt_ti, negative_prompt_ti])
            prompt_emb = None
            negative_prompt_emb = None

            compel = None
            del compel

        if torch.cuda.is_available():
            if xformers_memory_efficient_attention:
                self.pipe.enable_xformers_memory_efficient_attention()
            else:
                self.pipe.disable_xformers_memory_efficient_attention()

        try:
            self.pipe.scheduler = self.get_scheduler(sampler)
        except:
                print("Error in sampler, please try again")
                self.pipe = None
                self.base_model_id = None
                torch.cuda.empty_cache()
                gc.collect()
                return

        self.pipe.safety_checker = None

        # Get Control image
        if self.task_name != 'txt2img':
            if isinstance(image, str):
                # If the input is a string (file path), open it as an image
                image_pil = Image.open(image)
                numpy_array = np.array(image_pil, dtype=np.uint8)
            elif isinstance(image, Image.Image):
                # If the input is already a PIL Image, convert it to a NumPy array
                numpy_array = np.array(image, dtype=np.uint8)
            elif isinstance(image, np.ndarray):
                # If the input is a NumPy array, np.uint8
                numpy_array = image.astype(np.uint8)
            else:
                self.pipe = None
                self.base_model_id = None
                torch.cuda.empty_cache()
                gc.collect()
                if gui_active:
                    print("To use this function, you have to upload an image in the cell below first 👇")
                    return
                else:
                    raise ValueError("Unsupported image type or not control image found")

            # Extract the RGB channels
            try:
                array_rgb = numpy_array[:, :, :3]
            except:
                print("Unsupported image type")
                self.pipe = None
                self.base_model_id = None
                torch.cuda.empty_cache()
                gc.collect()
                raise ValueError("Unsupported image type") # return

        # Get params preprocess
        preprocess_params_config = {}
        if self.task_name != 'txt2img' and self.task_name != 'Inpaint':
            print("resolution is control_image(min(W, H))")
            preprocess_params_config["image"] = array_rgb
            preprocess_params_config["image_resolution"] = image_resolution # min(img_height.value, img_width.value)
            #preprocess_params_config["additional_prompt"] = additional_prompt # ""

            if self.task_name != "ip2p" and self.task_name != "Canny":
                if  self.task_name != "shuffle":
                    preprocess_params_config["preprocess_resolution"] = preprocess_resolution
                if self.task_name != "MLSD":
                    preprocess_params_config["preprocessor_name"] = preprocessor_name

        # RUN Preprocess
        if self.task_name == 'Inpaint':

            # Get mask for Inpaint
            if gui_active or os.path.exists(image_mask):
                # Read image mask from gui
                mask_control_img = Image.open(image_mask)
                numpy_array_mask = np.array(mask_control_img, dtype=np.uint8)
                array_rgb_mask = numpy_array_mask[:, :, :3]
            elif not gui_active:
                # Convert control image for draw
                name_without_extension = os.path.splitext(image.split('/')[-1])[0]
                image64 = base64.b64encode(open(image, 'rb').read())
                image64 = image64.decode('utf-8')
                img = np.array(plt.imread(f'{image}')[:,:,:3])

                # Create mask interactive
                draw(image64, filename=f"./{name_without_extension}_draw.png", w=img.shape[1], h=img.shape[0], line_width=0.04*img.shape[1])

                # Create mask and save
                with_mask = np.array(plt.imread(f"./{name_without_extension}_draw.png")[:,:,:3])
                mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
                plt.imsave(f"./{name_without_extension}_mask.png",mask, cmap='gray')
                mask_control = f"./{name_without_extension}_mask.png"
                print(f'Mask saved: {mask_control}')

                # Read image mask
                mask_control_img = Image.open(mask_control)
                numpy_array_mask = np.array(mask_control_img, dtype=np.uint8)
                array_rgb_mask = numpy_array_mask[:, :, :3]
            else:
                raise ValueError("No images found")

            init_image, control_mask, control_image = self.process_inpaint(
                image=array_rgb,
                image_resolution=image_resolution, ### edit
                preprocess_resolution=preprocess_resolution, # edit size in Inpaint
                image_mask=array_rgb_mask,
            )

        elif self.task_name == 'Openpose':
            print('Openpose')
            control_image = self.process_openpose(**preprocess_params_config)

        elif self.task_name == 'Canny':
            print('Canny')
            control_image = self.process_canny(
                **preprocess_params_config,
                low_threshold=low_threshold,
                high_threshold=high_threshold,
            )

        elif self.task_name == 'MLSD':
            print('MLSD')
            control_image = self.process_mlsd(
                **preprocess_params_config,
                value_threshold=value_threshold,
                distance_threshold=distance_threshold,
            )

        elif self.task_name == 'scribble':
            print('Scribble')
            control_image = self.process_scribble(**preprocess_params_config)

        elif self.task_name == 'softedge':
            print('Softedge')
            control_image = self.process_softedge(**preprocess_params_config)

        elif self.task_name == 'segmentation':
            print('Segmentation')
            control_image = self.process_segmentation(**preprocess_params_config)

        elif self.task_name == 'depth':
            print('Depth')
            control_image = self.process_depth(**preprocess_params_config)

        elif self.task_name == 'NormalBae':
            print('NormalBae')
            control_image = self.process_normal(**preprocess_params_config)

        elif 'lineart' in self.task_name:
            print('Lineart')
            control_image = self.process_lineart(**preprocess_params_config)

        elif self.task_name == 'shuffle':
            print('Shuffle')
            control_image = self.process_shuffle(**preprocess_params_config)

        elif self.task_name == 'ip2p':
            print('Ip2p')
            control_image = self.process_ip2p(**preprocess_params_config)


        # Get params for TASK
        pipe_params_config = {
            "prompt": None, #prompt, #self.get_prompt(prompt, additional_prompt),
            "negative_prompt": None, #negative_prompt,
            "prompt_embeds": prompt_emb,
            "negative_prompt_embeds": negative_prompt_emb,
            "num_images": num_images,
            "num_steps": num_steps,
            "guidance_scale": guidance_scale,
            "clip_skip": None, #clip_skip, because we use clip skip of compel
        }

        if not os.path.exists(self.base_model_id):
            # pipe_params_config["prompt_embeds"]                 = conditioning[0:1]
            # pipe_params_config["pooled_prompt_embeds"]          = pooled[0:1],
            # pipe_params_config["negative_prompt_embeds"]        = conditioning[1:2]
            # pipe_params_config["negative_pooled_prompt_embeds"] = pooled[1:2],
            # pipe_params_config["conditioning"]                  = conditioning,
            # pipe_params_config["pooled"]                        = pooled,
            # pipe_params_config["height"]                        = img_height
            # pipe_params_config["width"]                         = img_width
            pass
        elif self.task_name == 'txt2img':
            pipe_params_config["height"]                        = img_height
            pipe_params_config["width"]                         = img_width
        elif self.task_name == "Inpaint":
            pipe_params_config["strength"]                      = strength
            pipe_params_config["init_image"]                    = init_image
            pipe_params_config["control_mask"]                  = control_mask
            pipe_params_config["control_image"]                 = control_image
            pipe_params_config["controlnet_conditioning_scale"] = controlnet_conditioning_scale
            pipe_params_config["control_guidance_start"]        = control_guidance_start
            pipe_params_config["control_guidance_end"]          = control_guidance_end
        elif self.task_name != 'txt2img' and self.task_name != 'Inpaint':
            pipe_params_config["control_image"]                 = control_image
            pipe_params_config["controlnet_conditioning_scale"] = controlnet_conditioning_scale
            pipe_params_config["control_guidance_start"]        = control_guidance_start
            pipe_params_config["control_guidance_end"]          = control_guidance_end

        ### RUN PIPE ###
        for i in range (loop_generation):

            calculate_seed = random.randint(0, 2147483647) if seed == -1 else seed
            if generator_in_cpu:
                pipe_params_config["generator"] = torch.Generator().manual_seed(calculate_seed)
            else:
                try:
                    pipe_params_config["generator"] = torch.Generator("cuda").manual_seed(calculate_seed)
                except:
                    print("Generator in CPU")
                    pipe_params_config["generator"] = torch.Generator().manual_seed(calculate_seed)

            if not os.path.exists(self.base_model_id):
                #images = self.run_pipe_SDXL(**pipe_params_config)
                images = self.pipe(
                    prompt = None,
                    negative_prompt = None,
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    height = img_height,
                    width = img_width,
                    num_inference_steps = num_steps,
                    guidance_scale = guidance_scale,
                    clip_skip = None,
                    num_images_per_prompt = num_images,
                    generator = pipe_params_config["generator"],
                ).images
            elif self.task_name == 'txt2img':
                images = self.run_pipe_SD(**pipe_params_config)
            elif self.task_name == "Inpaint":
                images = self.run_pipe_inpaint(**pipe_params_config)
            elif self.task_name != 'txt2img' and self.task_name != 'Inpaint':
                results = self.run_pipe(**pipe_params_config) ## pipe ControlNet add condition_weights
                images =  [control_image] + results
                del results

            torch.cuda.empty_cache()
            gc.collect()

            if loop_generation > 1:

                mediapy.show_images(images)
                #print(image_list)
                #del images
                time.sleep(1)

            # save image
            image_list = []
            metadata = [
                    prompt,
                    negative_prompt,
                    self.base_model_id,
                    self.vae_model,
                    num_steps,
                    guidance_scale,
                    sampler,
                    calculate_seed
            ]

            for image_ in images:
                image_path = save_pil_image_with_metadata(image_, "./images", metadata)
                image_list.append(image_path)

            if loop_generation > 1:
                torch.cuda.empty_cache()
                gc.collect()
                print(image_list)
            print(f"Seed:\n{calculate_seed}")



        # if select_lora1.value != "None":
        #     model.pipe.unfuse_lora()
        #     model.pipe.unload_lora_weights()
        # if select_lora2.value != "None" or select_lora3.value != "None":
        #     print('BETA: reload weights for lora')
        #     model.load_pipe(select_model.value, task_name=options_controlnet.value, vae_model = vae_model_dropdown.value, reload=True)
        if xformers_memory_efficient_attention and torch.cuda.is_available():
            self.pipe.disable_xformers_memory_efficient_attention()
        self.pipe.to(self.device)
        self.pipe = self.process_lora(lora_A, lora_scale_A, unload=True)
        self.pipe = self.process_lora(lora_B, lora_scale_B, unload=True)
        self.pipe = self.process_lora(lora_C, lora_scale_C, unload=True)
        self.pipe = self.process_lora(lora_D, lora_scale_D, unload=True)
        self.pipe = self.process_lora(lora_E, lora_scale_E, unload=True)
        # model.pipe.unfuse_lora()
        # model.pipe.unload_lora_weights()

        torch.cuda.empty_cache()
        gc.collect()

        return images, image_list


# =====================================
# Prompt weights
# =====================================
from compel import Compel
from diffusers import StableDiffusionPipeline, DDIMScheduler
import re
from compel import EmbeddingsProvider, ReturnedEmbeddingsType

def concat_tensor(t):
    t_list = torch.split(t, 1, dim=0)
    t = torch.cat(t_list, dim=1)
    return t

def merge_embeds(prompt_chanks, compel):
    num_chanks = len(prompt_chanks)
    power_prompt = 1/(num_chanks*(num_chanks+1)//2)
    prompt_embs = compel(prompt_chanks)
    t_list = list(torch.split(prompt_embs, 1, dim=0))
    for i in range(num_chanks):
        t_list[-(i+1)] = t_list[-(i+1)] * ((i+1)*power_prompt)
    prompt_emb = torch.stack(t_list, dim=0).sum(dim=0)
    return prompt_emb

def detokenize(chunk, actual_prompt):
    chunk[-1] = chunk[-1].replace('</w>', '')
    chanked_prompt = ''.join(chunk).strip()
    while '</w>' in chanked_prompt:
        if actual_prompt[chanked_prompt.find('</w>')] == ' ':
            chanked_prompt = chanked_prompt.replace('</w>', ' ', 1)
        else:
            chanked_prompt = chanked_prompt.replace('</w>', '', 1)
    actual_prompt = actual_prompt.replace(chanked_prompt,'')
    return chanked_prompt.strip(), actual_prompt.strip()

def tokenize_line(line, tokenizer): # split into chunks
    actual_prompt = line.lower().strip()
    if actual_prompt == "":
      actual_prompt = 'worst quality'
    actual_tokens = tokenizer.tokenize(actual_prompt)
    max_tokens = tokenizer.model_max_length - 2
    comma_token = tokenizer.tokenize(',')[0]

    chunks = []
    chunk = []
    for item in actual_tokens:
        chunk.append(item)
        if len(chunk) == max_tokens:
            if chunk[-1] != comma_token:
                for i in range(max_tokens-1, -1, -1):
                    if chunk[i] == comma_token:
                        actual_chunk, actual_prompt = detokenize(chunk[:i+1], actual_prompt)
                        chunks.append(actual_chunk)
                        chunk = chunk[i+1:]
                        break
                else:
                    actual_chunk, actual_prompt = detokenize(chunk, actual_prompt)
                    chunks.append(actual_chunk)
                    chunk = []
            else:
                actual_chunk, actual_prompt = detokenize(chunk, actual_prompt)
                chunks.append(actual_chunk)
                chunk = []
    if chunk:
        actual_chunk, _ = detokenize(chunk, actual_prompt)
        chunks.append(actual_chunk)

    return chunks

def prompt_weight_conversor(input_string):
    # Convert prompt weights from a1... to comel

    # Find and replace instances of the colon format with the desired format
    converted_string = re.sub(r'\(([^:]+):([\d.]+)\)', r'(\1)\2', input_string)

    # Find and replace square brackets with round brackets and assign weight
    converted_string = re.sub(r'\[([^:\]]+)\]', r'(\1)0.909090909', converted_string)

    # Handle the general case of [x:number] and convert it to (x)0.9
    converted_string = re.sub(r'\[([^:]+):[\d.]+\]', r'(\1)0.9', converted_string)

    # Add a '+' sign after the closing parenthesis if no weight is specified
    modified_string = re.sub(r'\(([^)]+)\)(?![\d.])', r'(\1)+', converted_string)

    # double (())
    #modified_string = re.sub(r'\(\(([^)]+)\)\+\)', r'(\1)++', modified_string)

    # triple ((()))
    #modified_string = re.sub(r'\(\(([^)]+)\)\+\+\)', r'(\1)+++', modified_string)

    #print(modified_string)
    return modified_string

# =====================================
# IMAGE: METADATA AND SAVE
# =====================================
import os
from PIL import Image
from PIL.PngImagePlugin import PngInfo

def save_pil_image_with_metadata(image, folder_path, metadata_list):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    existing_files = os.listdir(folder_path)

    # Determine the next available image name
    image_name = f"image{str(len(existing_files) + 1).zfill(3)}.png"
    image_path = os.path.join(folder_path, image_name)

    try:
        # metadata
        metadata = PngInfo()
        metadata.add_text("Prompt", str(metadata_list[0]))
        metadata.add_text("Negative prompt", str(metadata_list[1]))
        metadata.add_text("Model", str(metadata_list[2]))
        metadata.add_text("VAE", str(metadata_list[3]))
        metadata.add_text("Steps", str(metadata_list[4]))
        metadata.add_text("CFG", str(metadata_list[5]))
        metadata.add_text("Scheduler", str(metadata_list[6]))
        metadata.add_text("Seed", str(metadata_list[7]))

        image.save(image_path, pnginfo=metadata)
    except:
        print('Saving image without metadata')
        image.save(image_path)

    return image_path

# =====================================
# LoRA Loaders
# =====================================
import torch
from safetensors.torch import load_file
from collections import defaultdict
def load_lora_weights(pipeline, checkpoint_path, multiplier, device, dtype):
    LORA_PREFIX_UNET = "lora_unet"
    LORA_PREFIX_TEXT_ENCODER = "lora_te"
    # load LoRA weight from .safetensors
    if isinstance(checkpoint_path, str):

        state_dict = load_file(checkpoint_path, device=device)

        updates = defaultdict(dict)
        for key, value in state_dict.items():
            # it is suggested to print out the key, it usually will be something like below
            # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

            layer, elem = key.split('.', 1)
            updates[layer][elem] = value

        # directly update weight in diffusers model
        for layer, elems in updates.items():

            if "text" in layer:
                layer_infos = layer.split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
                curr_layer = pipeline.text_encoder
            else:
                layer_infos = layer.split(LORA_PREFIX_UNET + "_")[-1].split("_")
                curr_layer = pipeline.unet

            # find the target layer
            temp_name = layer_infos.pop(0)
            while len(layer_infos) > -1:
                try:
                    curr_layer = curr_layer.__getattr__(temp_name)
                    if len(layer_infos) > 0:
                        temp_name = layer_infos.pop(0)
                    elif len(layer_infos) == 0:
                        break
                except Exception:
                    if len(temp_name) > 0:
                        temp_name += "_" + layer_infos.pop(0)
                    else:
                        temp_name = layer_infos.pop(0)

            # get elements for this layer
            weight_up = elems['lora_up.weight'].to(dtype)
            weight_down = elems['lora_down.weight'].to(dtype)
            alpha = elems['alpha']
            if alpha:
                alpha = alpha.item() / weight_up.shape[1]
            else:
                alpha = 1.0

            # update weight
            if len(weight_up.shape) == 4:
                curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up.squeeze(3).squeeze(2), weight_down.squeeze(3).squeeze(2)).unsqueeze(2).unsqueeze(3)
            else:
                curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up, weight_down)
    else:
        for ckptpath in checkpoint_path:
            state_dict = load_file(ckptpath, device=device)

            updates = defaultdict(dict)
            for key, value in state_dict.items():
                # it is suggested to print out the key, it usually will be something like below
                # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

                layer, elem = key.split('.', 1)
                updates[layer][elem] = value

            # directly update weight in diffusers model
            for layer, elems in updates.items():

                if "text" in layer:
                    layer_infos = layer.split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
                    curr_layer = pipeline.text_encoder
                else:
                    layer_infos = layer.split(LORA_PREFIX_UNET + "_")[-1].split("_")
                    curr_layer = pipeline.unet

                # find the target layer
                temp_name = layer_infos.pop(0)
                while len(layer_infos) > -1:
                    try:
                        curr_layer = curr_layer.__getattr__(temp_name)
                        if len(layer_infos) > 0:
                            temp_name = layer_infos.pop(0)
                        elif len(layer_infos) == 0:
                            break
                    except Exception:
                        if len(temp_name) > 0:
                            temp_name += "_" + layer_infos.pop(0)
                        else:
                            temp_name = layer_infos.pop(0)

                # get elements for this layer
                weight_up = elems['lora_up.weight'].to(dtype)
                weight_down = elems['lora_down.weight'].to(dtype)
                alpha = elems['alpha']
                if alpha:
                    alpha = alpha.item() / weight_up.shape[1]
                else:
                    alpha = 1.0

                # update weight
                if len(weight_up.shape) == 4:
                    curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up.squeeze(3).squeeze(2), weight_down.squeeze(3).squeeze(2)).unsqueeze(2).unsqueeze(3)
                else:
                    curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up, weight_down)
    return pipeline

def lora_mix_load(pipe, lora_path, alpha_scale=1.0, device='cuda', dtype=torch.float16):
    try:
        pipe=load_lora_weights(pipe, [lora_path], alpha_scale, device=device, dtype=torch.float16)
    except:
        pipe.load_lora_weights(lora_path)
        pipe.fuse_lora(lora_scale=alpha_scale)

    return pipe


# =====================================
# Inpainting canvas
# =====================================
canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

import base64, os
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
from shutil import copyfile
import shutil
import matplotlib.pyplot as plt

def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

# the control image of init_image and mask_image
def make_inpaint_condition(image, image_mask):
    image = np.array(image.convert("RGB")).astype(np.float32) / 255.0
    image_mask = np.array(image_mask.convert("L")).astype(np.float32) / 255.0

    assert image.shape[0:1] == image_mask.shape[0:1], "image and image_mask must have the same image size"
    image[image_mask > 0.5] = -1.0  # set as masked pixel
    image = np.expand_dims(image, 0).transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return image

# =====================================
# Adetailer
# =====================================
from functools import partial
from diffusers import DPMSolverMultistepScheduler, DPMSolverSinglestepScheduler
from asdff import AdCnPipeline, AdPipeline, yolo_detector
from huggingface_hub import hf_hub_download

def ad_model_process(
    face_detector_ad,
    person_detector_ad,
    hand_detector_ad,
    model_repo_id,
    common,
    inpaint,
    image_list_task,
    ):

    pipe = AdPipeline.from_pretrained(model_repo_id, torch_dtype=torch.float16)

    try:
        pipe.scheduler = DPMSolverSinglestepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True)
    except:
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

    pipe.safety_checker = None
    pipe.to("cuda")

    image_list_ad = []

    for path in image_list_task:
        if os.path.exists(path):
            # Open the image using PIL and convert it to PIL.Image.Image
            with Image.open(path) as img:
                images_ad = img.convert("RGB")

        detectors = []
        if face_detector_ad:
            face_model_path = hf_hub_download("Bingsu/adetailer", "face_yolov8n.pt")
            face_detector = partial(yolo_detector, model_path=face_model_path)
            detectors.append(face_detector)
        if person_detector_ad:
            person_model_path = hf_hub_download("Bingsu/adetailer", "person_yolov8s-seg.pt")
            person_detector = partial(yolo_detector, model_path=person_model_path)
            detectors.append(person_detector)
        if hand_detector_ad:
            hand_model_path = hf_hub_download("Bingsu/adetailer", "hand_yolov8n.pt")
            hand_detector = partial(yolo_detector, model_path=hand_model_path)
            detectors.append(hand_detector)

        result_ad = pipe(images=[images_ad], common=common, inpaint_only=inpaint, detectors=detectors)

        try:
            mediapy.show_images([result_ad[0][0], result_ad[1][0]])
        except:
            del pipe
            torch.cuda.empty_cache()
            gc.collect()
            return image_list_task

        image_path = save_pil_image_with_metadata(result_ad[0][0], f'{os.getcwd()}/images', metadata_list=None)
        image_list_ad.append(image_path)

    del pipe
    torch.cuda.empty_cache()
    gc.collect()

    return image_list_ad

In [ ]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Prompt** - Description of the image
#@markdown - **Negative Prompt** - Things you don't want to see or ignore in the image
#@markdown - **Steps** - Number of denoising steps. Higher steps may lead to better results but takes longer time to generate the image. Default is `30`.
#@markdown - **CFG** - Guidance scale ranging from `0` to `20`. Lower values allow the AI to be more creative and less strict at following the prompt. Default is `7.5`.
#@markdown - **Sampler** - The scheduler is responsible for controlling the learning rate of the diffusion model. Different schedulers can produce different results, so it is important to choose a scheduler that is appropriate for the desired task.
#@markdown - **Seed** - A random value that controls image generation. The same seed and prompt produce the same images. Set `-1` for using random seed values.
#@markdown - **Prompt weights** -  Are a way to control the influence of different text prompts on the image generation process. Prompt weights can be used to emphasize or de-emphasize certain aspects of the image, such as the object, the scene, or the style. Currently, the [Compel syntax](https://github.com/damian0815/compel/blob/main/doc/syntax.md) is being used. You can also activate the `Convert Prompt weights` to automatically convert syntax from `(word:1.1)` to `(word)1.1` or `(word)` to `(word)+` to make them compatible with Compel weights. Compel scale more with its values so that fewer weights are needed for good results
#@markdown - **ControlNet** - Is a neural network model for controlling diffusion models. It allows users to input additional information, such as edge maps, segmentation maps, and key points, into diffusion models to guide the image generation process.
#@markdown ---
%cd /content
import ipywidgets as widgets, mediapy, random
from diffusers.models.attention_processor import AttnProcessor2_0
from PIL import Image
import IPython.display
import time
from IPython.utils import capture
import logging
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, BoundedFloatText
logging.getLogger("diffusers").setLevel(logging.ERROR)

#from IPython.display import display
from ipywidgets import TwoByTwoLayout, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, interactive, HBox, VBox, BoundedIntText, BoundedFloatText

#PARAMETER WIDGETS
width = "225px"
auto_layout = widgets.Layout(height='auto', width='auto')
lora_layout = {'width':'165px'}
lora_scale_layout = {'width':'50px'}
style = {'description_width': 'initial'} # show full text

# =====================================
# Left
# =====================================

num_images = widgets.BoundedIntText(
    value = 1,
    min = 1,
    description="Images:",
    layout=widgets.Layout(width=width)
)

steps = widgets.BoundedIntText(
    value = 30,
    min = 1,
    max = 100,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.BoundedFloatText(
    value = 7.5,
    min = 0,
    step=0.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

select_sampler = widgets.Dropdown(
    options=[
        "DPM++ 2M",
        "DPM++ 2M Karras",
        "DPM++ 2M SDE",
        "DPM++ 2M SDE Karras",
        "DPM++ SDE",
        "DPM++ SDE Karras",
        "DPM2",
        "DPM2 Karras",
        "Euler",
        "Euler a",
        "Heun",
        "LMS",
        "LMS Karras",
        "DDIMScheduler",
        "DEISMultistepScheduler",
        "UniPCMultistepScheduler",
    ],
    description="Sampler:",
    layout=widgets.Layout(width=width),
)

img_height = widgets.BoundedIntText(
    min=64,
    max=4096,
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width),
)

img_width = widgets.BoundedIntText(
    min=64,
    max=4096,
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width),
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

#lora1
select_lora1 = widgets.Dropdown(
    options=lora_model_list,
    description="Lora1:",
    layout=lora_layout
)

lora_weights_scale1 = widgets.FloatText(
    min=-2.0,
    max=2.0,
    step=0.01,
    value=1,
    #description="Lora scale1:",
    layout=lora_scale_layout
)
#lora2
select_lora2 = widgets.Dropdown(
    options=lora_model_list,
    description="Lora2:",
    layout=lora_layout
)

lora_weights_scale2 = widgets.FloatText(
    min=-2.0,
    max=2.0,
    step=0.01,
    value=1,
    #description="Lora scale2:",
    layout=lora_scale_layout
)
#lora3
select_lora3 = widgets.Dropdown(
    options=lora_model_list,
    description="Lora3:",
    layout=lora_layout
)

lora_weights_scale3 = widgets.FloatText(
    min=-2.0,
    max=2.0,
    step=0.01,
    value=1,
    #description="Lora scale3:",
    layout=lora_scale_layout
)
#lora4
select_lora4 = widgets.Dropdown(
    options=lora_model_list,
    description="Lora4:",
    layout=lora_layout
)

lora_weights_scale4 = widgets.FloatText(
    min=-2.0,
    max=2.0,
    step=0.01,
    value=1,
    #description="Lora scale4:",
    layout=lora_scale_layout
)
#lora5
select_lora5 = widgets.Dropdown(
    options=lora_model_list,
    description="Lora5:",
    layout=lora_layout
)

lora_weights_scale5 = widgets.FloatText(
    min=-2.0,
    max=2.0,
    step=0.01,
    value=1,
    #description="Lora scale5:",
    layout=lora_scale_layout
)

select_clip_skip = widgets.Checkbox(
    value=True,
    description='Layer 2 Clip Skip',
    layout=widgets.Layout(width=width),
)

# =====================================
# Center
# =====================================

display_imgs = widgets.Output() # layout={'border': '1px solid black'}

select_model = widgets.Dropdown(
    options=model_list,
    description="Model:",
    layout=auto_layout,
)

vae_model_dropdown = widgets.Dropdown(
    options=vae_model_list,
    description="VAE:",
    layout=auto_layout,
)

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    rows=5,
    layout=auto_layout,
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    rows=5,
    layout=auto_layout,
)

active_ti = widgets.Checkbox(
    value=False,
    description='Active Textual Inversion in prompt (Experimental)',
    #style = style,
    layout=auto_layout,
)
# alternative prompt weights
weights_prompt = widgets.Checkbox(
    value=False,
    description='Convert Prompt weights',
    #style = style,
    layout=auto_layout,
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary",
    layout=Layout(height='auto', width='auto'),
)

### GENERATE ###

def generate_img(i):
  global model
  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  with display_imgs:

    print("Running...")


    # First load
    try:
        model
    except:
        model = Model(base_model_id=select_model.value, task_name=select_task.value, vae_model = vae_model_dropdown.value, type_model_precision = model_precision.value)

    model.load_pipe(select_model.value, task_name=select_task.value, vae_model = vae_model_dropdown.value, type_model_precision = model_precision.value)

    display_imgs.clear_output()

    try:
      preprocessor_name_found = int_inputs[select_task.value][0].value
    except:
      preprocessor_name_found = None

    global destination_path_cn_img, mask_control, image_list

    image_control_base = None
    if select_task.value != "txt2img":
        try:
            image_control_base = destination_path_cn_img
        except:
            print("No control image found")
            generate.disabled = False
            return

    mask_control_base = None
    if select_task.value == "Inpaint":
        if os.path.exists(int_inputs['Inpaint'][1].value):
            mask_control_base = int_inputs['Inpaint'][1].value
        else:
            try:
                mask_control_base = mask_control
            except:
                print("No mask image found")
                generate.disabled = False
                return

    pipe_params = {
    "prompt": prompt.value,
    "negative_prompt": neg_prompt.value,
    "img_height": img_height.value,
    "img_width": img_width.value,
    "num_images": num_images.value,
    "num_steps": steps.value,
    "guidance_scale": CFG.value,
    "clip_skip": select_clip_skip.value,
    "seed": random_seed.value,
    "image": image_control_base,
    "preprocessor_name": preprocessor_name_found,
    "preprocess_resolution": preprocess_resolution_global.value,
    "image_resolution": min(img_height.value, img_width.value), # min
    "additional_prompt": "",
    "image_mask": mask_control_base,
    "strength": int_inputs['Inpaint'][0].value, # only for Inpaint
    "low_threshold": int_inputs['Canny'][0].value,
    "high_threshold": int_inputs['Canny'][1].value,
    "value_threshold": int_inputs['MLSD'][0].value,
    "distance_threshold": int_inputs['MLSD'][1].value,
    "lora_A": select_lora1.value,
    "lora_scale_A": lora_weights_scale1.value,
    "lora_B": select_lora2.value,
    "lora_scale_B": lora_weights_scale2.value,
    "lora_C": select_lora3.value,
    "lora_scale_C": lora_weights_scale3.value,
    "lora_D": select_lora4.value,
    "lora_scale_D": lora_weights_scale4.value,
    "lora_E": select_lora5.value,
    "lora_scale_E": lora_weights_scale5.value,
    "active_textual_inversion": active_ti.value,
    "textual_inversion": embed_list,
    "convert_weights_prompt": weights_prompt.value,
    "sampler": select_sampler.value,
    "xformers_memory_efficient_attention": xformers_memory_efficient_attention.value,
    "gui_active": True,
    "loop_generation": loop_generator.value,
    "controlnet_conditioning_scale" : controlnet_output_scaling_in_unet.value,
    "control_guidance_start" : controlnet_start_threshold.value,
    "control_guidance_end" : controlnet_stop_threshold.value,
    "generator_in_cpu" : init_generator_in_cpu.value,
    }

    images, image_list = model(**pipe_params)

    if loop_generator.value == 1:
        mediapy.show_images(images)
        del images

    torch.cuda.empty_cache()
    gc.collect()

  generate.disabled = False
  return

generate.on_click(generate_img)

show_textual_inversion = widgets.Button(
    description="List available textual inversions",
    disabled=False,
    button_style="info",
    layout=widgets.Layout(height='auto', width='auto'),
)

def elemets_textual_inversion(value):
  with display_imgs:
    print('The embeddings currently supported. Write in the prompt the word for use')
    for name, directory_name in embed_list:
        print(name)
    return

show_textual_inversion.on_click(elemets_textual_inversion)

clear_outputs = widgets.Button(
    description="Clear outputs",
    disabled=False,
    button_style="info",
    layout=widgets.Layout(height='auto', width='auto'),

)

def clear_outputs_run(value):
  display_imgs.clear_output()
  return

clear_outputs.on_click(clear_outputs_run)

# =====================================
# Right ControlNet
# =====================================

control_model_list = list(CONTROLNET_MODEL_IDS.keys())

# Create a Dropdown for selecting options
select_task = widgets.Dropdown( #
    options=[
        control_model_list[13],
        control_model_list[12],
        control_model_list[0],
        control_model_list[1],
        control_model_list[2],
        control_model_list[3],
        control_model_list[4],
        control_model_list[5],
        control_model_list[6],
        control_model_list[7],
        control_model_list[8],
        control_model_list[10],
        control_model_list[11],
    ],
    description='TASK:',
    layout=auto_layout,
)

controlnet_output_scaling_in_unet = widgets.FloatText(value=1.0, min=0.0, max=5.0, step=0.1, description='ControlNet Output Scaling in UNet:', style=style)
controlnet_start_threshold = widgets.FloatSlider(value=0.0, min=0.00, max=1.0, step=0.01, description='ControlNet Start Threshold (%):', style=style)
controlnet_stop_threshold = widgets.FloatSlider(value=1.0, min=0.00, max=1.0, step=0.01, description='ControlNet Stop Threshold (%):', style=style)

preprocess_resolution_global = widgets.IntSlider(
    value=512,
    min=256,
    max=2048,
    description='Preprocess resolution',
    style=style,
)

# Create a dictionary to map options to lists of IntText widgets
int_inputs = {

    control_model_list[13]: [
    ],
    control_model_list[12]: [
        widgets.FloatSlider(value=1.0, min=0.01, max=1.0, step=0.01, description='Inpaint strength:', layout=Layout(visibility='hidden'), style=style),
        widgets.Text(value="", placeholder="/content/my_mask.png", rows=1, description='Mask path:', layout=Layout(visibility='hidden'), style=style)
    ],
    control_model_list[0]: [
        widgets.Dropdown(value='Openpose', description='Preprocessor:', options=['None','Openpose'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[1]: [
        widgets.BoundedIntText(value=100, min=1, max=255, description='Canny low threshold:', layout=Layout(visibility='hidden'), style=style),
        widgets.BoundedIntText(value=200, min=1, max=255, description='Canny high threshold:', layout=Layout(visibility='hidden'), style=style)
    ],
    control_model_list[2]: [
        widgets.BoundedFloatText(value=0.1, min=1, max=2.0, step=0.01, description='Hough value threshold (MLSD):', layout=Layout(visibility='hidden'), style=style),
        widgets.BoundedFloatText(value=0.1, min=1, max=20.0, step=0.01, description='Hough distance threshold (MLSD):', layout=Layout(visibility='hidden'), style=style)
    ],
    control_model_list[3]: [
        widgets.Dropdown(value='HED', description='Preprocessor:', options=['HED','PidiNet', 'None'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[4]: [
        widgets.Dropdown(value='PidiNet', description='Preprocessor:', options=['HED','PidiNet', 'HED safe', 'PidiNet safe','None'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[5]: [
        widgets.Dropdown(value='UPerNet', description='Preprocessor:', options=['UPerNet','None'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[6]: [
        widgets.Dropdown(value='DPT', description='Preprocessor:', options=['Midas', 'DPT','None'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[7]: [
        widgets.Dropdown(value='NormalBae', description='Preprocessor:', options=['NormalBae','None'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[8]: [
        widgets.Dropdown(value='Lineart', description='Preprocessor:', options=['Lineart','Lineart coarse', 'None', 'Lineart (anime)', 'None (anime)'], layout=Layout(visibility='hidden'), style=style),
    ],
    control_model_list[10]: [
        widgets.Dropdown(value='ContentShuffle', description='Preprocessor:', options=['ContentShuffle','None'], layout=Layout(visibility='hidden'), style=style),
    ],
}

# Function to update visibility and enable/disable state of widgets
def update_widgets(option):
    for opt, int_inputs_list in int_inputs.items():
        if opt == option:
            for int_input in int_inputs_list:
                int_input.layout.visibility = 'visible'
        else:
            for int_input in int_inputs_list:
                int_input.layout.visibility = 'hidden'

interactive(update_widgets, option=select_task)

# =====================================
# Settings
# =====================================
loop_generator = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    description='Loops 🔁',
    #layout=auto_layout,
    style = style,
)

xformers_memory_efficient_attention = widgets.Checkbox(
    value=True,
    description='Xformers memory efficient attention',
    layout=auto_layout,
    style = style,
)

model_precision = widgets.RadioButtons(
    options=[("float16", torch.float16),("float32", torch.float32)],
    description="Model precision (float32 need more memory):",
    layout=auto_layout,
    style = style,
)

init_generator_in_cpu = widgets.Checkbox(
    value=False,
    description='Generate initial random noise with the seed on the CPU',
    layout=auto_layout,
    style = style,
)
# =====================================
# App
# =====================================

title_tab_one = widgets.HTML(
    value="<h2>SD Interactive</h2>",
    layout=widgets.Layout(display="flex", justify_content="center")
)
title_tab_two = widgets.HTML(
    value="<h2>Settings</h2>",
    layout=widgets.Layout(display="flex", justify_content="center")
)

buttons_ = TwoByTwoLayout(top_left=generate,
               top_right=show_textual_inversion,
               #bottom_left=None,
               bottom_right=clear_outputs, merge=True)

show_textual_inversion.style.button_color = '#97BC62'
clear_outputs.style.button_color = '#97BC62'
generate.style.button_color = '#2C5F2D'

# TAB 1
tab_sd = widgets.VBox(
    [
      widgets.AppLayout(
        header=None,
        left_sidebar=widgets.VBox(
            [
                title_tab_one,
                num_images,
                steps,
                CFG,
                select_sampler,
                img_width,
                img_height,
                random_seed,
                widgets.HBox([select_lora1, lora_weights_scale1],layout=widgets.Layout(width=width)),
                widgets.HBox([select_lora2, lora_weights_scale2],layout=widgets.Layout(width=width)),
                widgets.HBox([select_lora3, lora_weights_scale3],layout=widgets.Layout(width=width)),
                widgets.HBox([select_lora4, lora_weights_scale4],layout=widgets.Layout(width=width)),
                widgets.HBox([select_lora5, lora_weights_scale5],layout=widgets.Layout(width=width)),
                select_clip_skip,
            ]
        ),
        center=widgets.VBox(
            [
                select_task,
                select_model,
                vae_model_dropdown,
                prompt,
                neg_prompt,
                widgets.HBox([weights_prompt ,active_ti]),
                buttons_,
            ]
        ),
        right_sidebar=widgets.VBox(
            [
                controlnet_output_scaling_in_unet,
                controlnet_start_threshold,
                controlnet_stop_threshold,
            ] +
            [preprocess_resolution_global] + [int_input for int_inputs_list in int_inputs.values() for int_input in int_inputs_list]
        ),
        footer=None,
        pane_widths=[1.1, 1.4, 1.2],
        # pane_heights=["0px", 1, '0px'],
      ),
      display_imgs,
    ]
)

# TAB 2
tab_settings = widgets.VBox([
    title_tab_two,
    loop_generator,
    xformers_memory_efficient_attention,
    init_generator_in_cpu,
    model_precision,
    widgets.HTML(
        value="<p>⏲️</p>",
        layout=widgets.Layout(display="flex", justify_content="center")
    ),
])

# APP
tab = widgets.Tab()
tab.children = [
  widgets.VBox(
    children = [
        tab_sd,
    ]),
  widgets.VBox(
    children = [
        tab_settings,
    ]),
]

tab.set_title(0, "Stable Diffusion")
tab.set_title(1,  "Settings")
tab.selected_index = 0

display(tab)

In [ ]:
#@title Upload an image here for use in Inpainting or ControlNet. 👈‍‍ 🖼️🖼️🖼️
#@markdown - To use Controlnet, you need to upload the control image with this cell
Create_mask_for_Inpaint = True # @param {type:"boolean"}
stroke_width = 24 # @param {type:"integer"}
from google.colab import files
from IPython.display import HTML
import os
import shutil
%cd /content
uploaded = files.upload()

filename = next(iter(uploaded))
print(f'Uploaded file: {filename}')

upload_folder = 'uploaded_controlnet_image/'
if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)

source_path = filename
destination_path_cn_img = os.path.join(upload_folder, filename)
shutil.move(source_path, destination_path_cn_img)
print(f'Moved file to {destination_path_cn_img}')

if select_task.value == 'Inpaint' or Create_mask_for_Inpaint:
    init_image = destination_path_cn_img
    name_without_extension = os.path.splitext(init_image.split('/')[-1])[0]

    image64 = base64.b64encode(open(init_image, 'rb').read())
    image64 = image64.decode('utf-8')

    print('\033[34m Draw the mask with the mouse \033[0m')
    img = np.array(plt.imread(f'{init_image}')[:,:,:3])

    draw(image64, filename=f"./{name_without_extension}_draw.png", w=img.shape[1], h=img.shape[0], line_width=stroke_width)

    with_mask = np.array(plt.imread(f"./{name_without_extension}_draw.png")[:,:,:3])
    mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
    plt.imsave(f"./{name_without_extension}_mask.png",mask, cmap='gray')
    mask_control = f"./{name_without_extension}_mask.png"
    print(f'\033[34m Mask saved: {mask_control} \033[0m')

In [ ]:
#@title 👇 Upscale and face restoration { form-width: "20%", display-mode: "form" }
from IPython.utils import capture
import os
import shutil

%cd /content
directory_codeformer = '/content/CodeFormer/'
with capture.capture_output() as cap:
  if not os.path.exists(directory_codeformer):
      os.makedirs(directory_codeformer)

      # Setup
      # Clone CodeFormer and enter the CodeFormer folder
      %cd /content
      !git clone https://github.com/sczhou/CodeFormer.git
      %cd CodeFormer


      # Set up the environment
      # Install python dependencies
      !pip install -q -r requirements.txt
      !pip -q install ffmpeg
      # Install basicsr
      !python basicsr/setup.py develop

      # Download the pre-trained model
      !python scripts/download_pretrained_models.py facelib
      !python scripts/download_pretrained_models.py CodeFormer
  del cap
# Visualization function
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# Copy imgs
Select_an_image = "" # @param {type:"string"}

# PROCESS AD
if os.path.exists(Select_an_image.strip()):
    image_list = [Select_an_image.replace('/content/', '').strip()]

destination_directory = '/content/CodeFormer/inputs/user_upload'
!rm -rf /content/CodeFormer/inputs/user_upload/*
os.makedirs(destination_directory, exist_ok=True)
for image_path in image_list:
    image_filename = os.path.basename('/content/'+image_path)
    destination_path = os.path.join(destination_directory, image_filename)
    try:
        shutil.copyfile('/content/'+image_path, destination_path)
        print(f"Image '{image_filename}' has been copied to '{destination_path}'")
    except Exception as e:
        print(f"Failed to copy '{image_filename}' to '{destination_path}': {e}")

#@markdown `CODEFORMER_FIDELITY`: Balance the quality (lower number) and fidelity (higher number)<br>
# you can add '--bg_upsampler realesrgan' to enhance the background
CODEFORMER_FIDELITY = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown `BACKGROUND_ENHANCE`: Enhance background image with Real-ESRGAN<br>
BACKGROUND_ENHANCE = False #@param {type:"boolean"}
#@markdown `FACE_UPSAMPLE`: Upsample restored faces for high-resolution AI-created images<br>
FACE_UPSAMPLE = False #@param {type:"boolean"}
#markdown `HAS_ALIGNED`: Input are cropped and aligned faces<br>
HAS_ALIGNED =  False
#@markdown `UPSCALE`: The final upsampling scale of the image. Default: 2<br>
UPSCALE = 2 #@param {type:"slider", min:2, max:8, step:1}
#markdown `DETECTION_MODEL`: Face detector. Default: retinaface_resnet50<br>
DETECTION_MODEL = "retinaface_resnet50"
#markdown `DRAW_BOX`: Draw the bounding box for the detected faces.
DRAW_BOX = False

BACKGROUND_ENHANCE = '--bg_upsampler realesrgan' if BACKGROUND_ENHANCE else ''
FACE_UPSAMPLE = '--face_upsample' if FACE_UPSAMPLE else ''
HAS_ALIGNED = '--has_aligned' if HAS_ALIGNED else ''
DRAW_BOX = '--draw_box' if DRAW_BOX else ''
%cd CodeFormer
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path {destination_directory} {BACKGROUND_ENHANCE} {FACE_UPSAMPLE} {HAS_ALIGNED} --upscale {UPSCALE} --detection_model {DETECTION_MODEL} {DRAW_BOX}


import os
import glob

input_folder = 'inputs/user_upload'
result_folder = f'results/user_upload_{CODEFORMER_FIDELITY}/final_results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  basename = os.path.splitext(os.path.basename(input_path))[0]
  output_path = os.path.join(result_folder, basename+'.png')
  img_output = imread(output_path)
  display(img_input, img_output)

%cd /content

Results in /content/CodeFormer/results


In [ ]:
#@title Download Images
import os
from google.colab import files
!rm /content/results.zip
!ls /content/images
print('Download results')
os.system(f'zip -r results.zip /content/images')
try:
  files.download("results.zip")
except:
  print("Error")

In [ ]:
#@title Download Upscale results
import os
from google.colab import files
import shutil
%cd /content/CodeFormer
!ls results
print('Download results')
os.system(f'zip -r results.zip results/user_upload_{CODEFORMER_FIDELITY}/final_results')
try:
  files.download("results.zip")
except:
  files.download(f'/content/CodeFormer/results/{filename[:-4]}_{CODEFORMER_FIDELITY}/{filename}')
%cd /content

# Extras

In [ ]:
# You can also use this cell to simply reload the model in case you need to.
del model

In [ ]:
from PIL import Image
import os

try:
  del model
  torch.cuda.empty_cache()
  gc.collect()
except:
  torch.cuda.empty_cache()
  gc.collect()

# CONVERT SAFETENSORS TO DIFFUSERS FOR USE HERE
# from diffusers import StableDiffusionPipeline
# pipe = StableDiffusionPipeline.from_single_file(select_model.value).to("cuda")
# pipe.save_pretrained("./adetailer_model/") # model path inpaint is ./adetailer_model/
# del pipe
# torch.cuda.empty_cache()
# gc.collect()

# OPTIONS #
# @markdown # Adetailer 🚫 beta
face_detector_ad = True # @param {type:"boolean"}
person_detector_ad = True # @param {type:"boolean"}
hand_detector_ad = False # @param {type:"boolean"}
model_for_inpaint = "frankjoshua/toonyou_beta6" # @param {type:"string"}
common = {
    "prompt": "masterpiece, best quality",
    "num_inference_steps": 40,
    "height": img_height.value,
    "width": img_width.value,
}
inpaint = {
    "prompt": "masterpiece, best quality",
    "num_inference_steps": 40,
    "height": img_height.value,
    "width": img_width.value,
}
Select_an_image = "" # @param {type:"string"}

# PROCESS AD
if os.path.exists(Select_an_image):
    image_list = [Select_an_image]

image_list = ad_model_process(
    face_detector_ad,
    person_detector_ad,
    hand_detector_ad,
    model_for_inpaint,
    common,
    inpaint,
    image_list,
)